In [1]:
import sys
import os

# Agrega la carpeta raíz del proyecto (src) al path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [37]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from datetime import datetime, timedelta
from meteostat import Point, Daily
from utils.utils import *
import requests

In [3]:
df_shark = pd.read_csv(r'..\data\shark_attack_clean.csv')
df_shark.drop(['Unnamed: 0', 'age'], axis=1, inplace=True)


In [79]:
import pandas as pd

# Cargar archivo
df = pd.read_csv(r"..\data\temperatura_mar.csv", sep='\s+', header=0)

# Opcional: limitar a años razonables
df = df[df["year"].between(1900, 2022)]


In [80]:
df.head()

,year,lat,lon,count,value
8754,1900,12.5,107.5,5,-0.09
8755,1900,12.5,122.5,1,-1.39
8756,1900,-12.5,132.5,2,0.02
8757,1900,-12.5,-172.5,2,0.34
8758,1900,12.5,-17.5,2,-1.24


In [ ]:
# Nueva columna con las decadas, para utilizar esta columna en los gráficos
df_shark['decade'] = (df_shark['year'] // 10 * 10).astype(int)

In [49]:
# Paso la columna date  a datetime y la columna year a int 
df_shark['date'] = pd.to_datetime(df_shark['date'], errors='coerce')
df_shark['year'] = df_shark['year'].astype(int) 

In [58]:
df_shark.loc[df_shark['activity2'] == 'Boogie boarding', 'activity2'] = 'Body boarding'

In [73]:
# Nueva columna diferenciancdo entre actividad de ocio y actividad no de ocio
df_shark['ocio_o_no'] = 'No ocio'
df_shark.loc[df_shark['activity2'].isin(['Surfing', 'Swimming', 'Snorkeling', 'Body boarding', 'Paddling', 'Diving', 
                                         'Kite Boarding', 'Playing', 'Rowing', 'Surf skiing', 'Paddle boarding',
                                         'Kayaking', 'Walking', 'Sitting on surfboard', 'Collecting', 'Canoeing']),
                                         'ocio_o_no'] = 'Ocio'
df_shark.loc[df_shark['activity2'].isin(['Random Activity', 'No details'])] = 'Desconocido'

In [ ]:
# Combierto zonas atacadas en puntos geolocalizables de latitud y longitud para poder hacer un mapa visual de los datos
df_shark['geolocalizacion'] = df_shark.apply(lambda row: geo_point(row, 'area', 'country'), axis=1)

In [65]:
df_shark_2 = df_shark[~df_shark['geolocalizacion'].isna()]
df_shark.shape, df_shark_2.shape

((5738, 14), (5257, 14))

In [75]:
df_shark_2['type'].value_counts()

type
Unprovoked      3935
Provoked         515
Invalid          398
Watercraft       287
Sea Disaster      85
Unverified        17
Questionable      15
Boat               5
Name: count, dtype: int64

In [91]:
df_shark.iloc[-1]

date               Desconocido
year               Desconocido
type               Desconocido
country            Desconocido
area               Desconocido
location           Desconocido
activity           Desconocido
name               Desconocido
sex                Desconocido
fatal_y_n          Desconocido
species            Desconocido
activity2          Desconocido
decade             Desconocido
geolocalizacion    Desconocido
ocio_o_no          Desconocido
Name: 5737, dtype: object

In [93]:
df_shark['lat'] = df_shark['geolocalizacion'].apply(lambda x: x[0] if x is not None else None)
df_shark['lon'] = df_shark['geolocalizacion'].apply(lambda x: x[1] if x is not None else None)

In [ ]:
df_shark['["lat", "lon"]'].dtypes

lat    object
lon    object
dtype: object

In [98]:
df_shark[df_shark['fatal_y_n'] == 'Desconocido'] = 'UNKNOWN'
df_shark[df_shark['fatal_y_n'] == 'Surf skiing'] = 'N'
df_shark[df_shark['fatal_y_n'] == 'F'] = 'Y'
df_shark[df_shark['fatal_y_n'] == 'n'] = 'N'
df_shark[df_shark['fatal_y_n'] == 'Nq'] = 'UNKNOWN'

In [99]:
df_shark['fatal_y_n'].unique()

array(['Y', 'N', 'UNKNOWN'], dtype=object)

In [100]:
df_shark.to_csv(r'C:\Users\Vicen\Visual Code\The_Bridge\Personal_2506_dsft_thebridge\2-Analytics\EDA\EDA-SHARK-ATTACK\src\data\shark_attack_enriquecido.csv')

In [101]:
df_shark.head()

,date,year,type,country,area,location,activity,name,sex,fatal_y_n,species,activity2,decade,geolocalizacion,ocio_o_no,lat,lon
0,2023-05-13 00:00:00,2023,Unprovoked,AUSTRALIA,South Australia,Elliston,Surfing,Simon Baccanello,M,Y,White shark,Surfing,2020,"(-30.5343665, 135.6301212)",Ocio,-30.534367,135.630121
1,2023-04-29 00:00:00,2023,Unprovoked,AUSTRALIA,Western Australia,"Yallingup, Busselton",Swimming,male,M,N,1m shark,Swimming,2020,"(-25.2303005, 121.0187246)",Ocio,-25.2303,121.018725
2,2022-10-07 00:00:00,2022,Unprovoked,AUSTRALIA,Western Australia,Port Hedland,Spearfishing,Robbie Peck,M,N,Bull shark,Fishing,2020,"(-25.2303005, 121.0187246)",No ocio,-25.2303,121.018725
3,2021-10-04 00:00:00,2021,Unprovoked,USA,Florida,"Fort Pierce State Park, St. Lucie County",Surfing,Truman Van Patrick,M,N,Unknow,Surfing,2020,"(27.7567667, -81.4639835)",Ocio,27.756767,-81.463983
4,2021-10-03 00:00:00,2021,Unprovoked,USA,Florida,"Jensen Beach, Martin County",Swimming,male,M,N,Unknow,Swimming,2020,"(27.7567667, -81.4639835)",Ocio,27.756767,-81.463983
